In [17]:
import sys
sys.path.append('..')
from pathlib import Path
from datetime import datetime
from dateutil import tz

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation
from matplotlib import colors
from metpy.plots.ctables import registry
from metpy.plots import colortables
from metpy.plots import USCOUNTIES
from metpy.units import units
from siphon.catalog import TDSCatalog
import xarray as xr

In [18]:
# Scan the catalog and download the data

def get_satellite(frame):
    satcat = TDSCatalog('https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/'
                        'CloudAndMoistureImagery/CONUS/Channel11/current/catalog.xml')
    dataset = satcat.datasets[frame]
    dataset = dataset.remote_access(use_xarray=True)
    return dataset

def get_goes_lightning_mapper(frame):
    cat = TDSCatalog('https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/'
                     'GeostationaryLightningMapper/CONUS/current/catalog.xml')
    dataset = cat.datasets[frame]
    dataset = dataset.remote_access(use_xarray=True)
    return dataset

# Convert UTC time to Eastern Time
def convert_datetime(dt):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New York')
    utc_time = datetime.strptime(dt, '%Y-%m-%dT%H:%M:%S').replace(tzinfo=from_zone)
    eastern = utc_time.astimezone(to_zone)
    date_time = datetime.strftime(eastern, '%a, %b %d, %Y %I:%M %p').lstrip('0').replace(' 0', ' ')
    return date_time

In [19]:
%%capture
# Store satellite and lightning time-series data in a tuple
datasets_glm = ()
    
for frame in range(240, -1, -15):
    ds = get_goes_lightning_mapper(frame)
    datasets_glm = (*datasets_glm, ds)
    

In [20]:
%%capture
# Grab first dataset and make figure using its projection data
data = datasets_glm[0].metpy.parse_cf('flash_extent_density')
proj = data.metpy.cartopy_crs

fig = plt.figure(figsize=(1920/72, 1080/72))
ax = fig.add_axes([0, 0, 1, 1], projection=proj)
ax.set_extent([-96.1, -70, 23.2, 37.5])
ax.add_feature(cfeature.OCEAN.with_scale('50m'), color='lightgray')
ax.add_feature(USCOUNTIES.with_scale('20m'), edgecolor='lightgray', linewidth=0.75)
ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=3.0)
ax.set_adjustable('datalim')
ax.outline_patch.set_visible(False)

In [21]:
# Container for the artists of all frames. Each item is a tuple of (image, text)
artists_glm = []

for ds_glm in datasets_glm:
    fed = ds_glm.metpy.parse_cf('flash_extent_density')
    fed_x = fed['x']
    fed_y = fed['y']
    
    # Create Banner
    banner = ax.text(s=f'GOES-16 Lightning Mapper Flash Density \nPast Few Hours', y=0.95, x=0.01, 
                     transform=ax.transAxes, zorder=100, fontname='Arial', color='white',
                     fontweight='bold', fontsize=48, va='top',
                     bbox=dict(facecolor='navy', alpha=1.0, edgecolor='none'))
    
    # Plot image
    im_fed = ax.imshow(fed, extent=(fed_x.min(), fed_x.max(), fed_y.min(), fed_y.max()),
                       origin='upper', vmin=1, vmax=6, cmap=plt.cm.YlOrBr)
    
    artists_glm.append((im_fed, banner))

In [22]:
# Build and save animation
graphics_dir = Path('.') / '..' / '..' / 'graphics' / 'satellite'
anim = ArtistAnimation(fig, artists_glm, interval=200., blit=False)
anim.save(f'{graphics_dir}/glm.gif', writer='pillow')
anim